In [9]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

In [10]:
# Load processed data
train = pd.read_csv('airbnb_train_processed.csv')
test = pd.read_csv('airbnb_test_processed.csv')

In [11]:
# Separate features and target
X_train = train.drop('price', axis=1).values
y_train = train['price'].values.reshape(-1, 1)
X_test = test.drop('price', axis=1).values
y_test = test['price'].values.reshape(-1, 1)

In [12]:
# --- 🔹 SCALE INPUTS AND TARGET ---
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [13]:
# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test_scaled = torch.tensor(y_test_scaled, dtype=torch.float32)

In [14]:
# Neural Network
class AirbnbNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

model = AirbnbNN(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [15]:
# Training loop
for epoch in range(200):  # more epochs
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 20, Loss: 0.8055
Epoch 40, Loss: 0.7095
Epoch 60, Loss: 0.6577
Epoch 80, Loss: 0.6111
Epoch 100, Loss: 0.5666
Epoch 120, Loss: 0.5261
Epoch 140, Loss: 0.4858
Epoch 160, Loss: 0.4437
Epoch 180, Loss: 0.4028
Epoch 200, Loss: 0.3651


In [16]:
# Evaluation
model.eval()
with torch.no_grad():
    pred_scaled = model(X_test)
    # --- 🔹 INVERSE TRANSFORM THE PREDICTIONS ---
    pred = scaler_y.inverse_transform(pred_scaled.numpy())
    y_true = y_test  # already original scale

    r2 = r2_score(y_true, pred)
    import numpy as np
    rmse = np.sqrt(mean_squared_error(y_true, pred))

print(f'✅ Neural Network R2: {r2:.4f}, RMSE: ${rmse:.2f}')

✅ Neural Network R2: 0.3427, RMSE: $242.23
